# Test with Feature extraction, no distractor, simple tree, 5 random pivots, cosine similarity

## Import libraries

In [2]:
!pip install scikit-learn-extra
from google.colab import drive
from pathlib import Path
from matplotlib import pyplot as plt 
import pandas as pd
import numpy as np
import os
import csv
import concurrent.futures
import sklearn_extra
from sklearn_extra.cluster import KMedoids
import time

     |████████████████████████████████| 1.7 MB 5.2 MB/s 


## Utility functions

### Create annot and load descriptors

In [3]:
def create_annot(path):
  image_list = list(Path(path).glob('*/*.jpg'))
  # the identity name is in the path (the name of the parent directory)
  names_list = [i.parent.name for i in image_list]  # get the identity of each image
  # keep info in a pandas DataFrame
  annot = pd.DataFrame({'identity': names_list, 'image_path': image_list})
  return annot

def concatenate_annots(list_of_paths):
  concat_annot = pd.DataFrame()
  with concurrent.futures.ThreadPoolExecutor() as executor:
    annots = [executor.submit(create_annot, path) for path in list_of_paths]
    for annot in annots:
      new_annot = annot.result()
      concat_annot = concat_annot.append(new_annot, ignore_index = True)
    return concat_annot

In [4]:
def load_descriptors(path):
  with open(path, 'rb') as file:
    return np.load(file)

def concatenate_descriptors(list_of_paths):
  concat_descriptors = None
  with concurrent.futures.ThreadPoolExecutor() as executor:
    descriptors = [executor.submit(load_descriptors, path) for path in list_of_paths]
    for descriptor in descriptors:
      new_descriptor = descriptor.result()
      if concat_descriptors is None:
        concat_descriptors = new_descriptor
      else:
        concat_descriptors = np.concatenate([concat_descriptors, new_descriptor])
    return concat_descriptors

### Create pivots

In [5]:
def generate_pivots(descriptors, n, strategy="kMED"):
  if strategy == "kMED":
    kmedoids = sklearn_extra.cluster.KMedoids(n_clusters=n).fit(descriptors)
    return kmedoids.cluster_centers_
  if strategy != "rnd":
    print(strategy, "was not implemented. Random pivots were returned")
  pivots_id = np.random.choice(np.arange(len(descriptors)), size=n)
  return descriptors[pivots_id]

def generate_list_of_pivots(descriptors, t, n, strategy="rnd"):
  list_of_pivots = []
  with concurrent.futures.ThreadPoolExecutor() as executor:
    pivots = [executor.submit(generate_pivots, descriptors, n, strategy) for i in range(t)]
    for pivot in concurrent.futures.as_completed(pivots):
      new_pivot = pivot.result()
      list_of_pivots.append(new_pivot)
    return list_of_pivots

### Save test results

In [6]:
def save_results(dir, file_name, results):
  with open(os.path.join(dir, file_name +".csv"), 'w') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(["CLASS", "AP", "QUERY TIME"])
    # write the data
    for r in results:
      writer.writerow(r) 

## Test Performance

In [7]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Create annot and load descriptors for the database

In [8]:
db_annot = create_annot('/content/drive/MyDrive/CV_Birds/train')
db_annot

,identity,image_path
0,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
1,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
2,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
3,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
4,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
...,...,...
47327,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/train/YELLOW H...
47328,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/train/YELLOW H...
47329,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/train/YELLOW H...
47330,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/train/YELLOW H...


In [9]:
db_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/training/ResNet50v2/resnet_fe_features.npy')
db_descriptors.shape

(47332, 2048)

### Create annot and load descriptors for the test set

In [10]:
query_annot = create_annot('/content/drive/MyDrive/CV_Birds/test')
query_annot

,identity,image_path
0,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
1,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
2,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
3,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
4,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
...,...,...
1620,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...
1621,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...
1622,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...
1623,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...


In [11]:
query_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/test/ResNet50v2/resnet50_fe_features.npy')
query_descriptors.shape

(1625, 2048)

To run our tests we select only the first image of each species within the test set. Please note that within the test set we have 5 images per species.

In [12]:
queries_indexes = [x for x in range(325*5) if x%5 == 0]

### Compute mAP

In [13]:
birds_db = db_annot
counts = birds_db.groupby('identity').count()
print("Minimum number of images per species:", int(counts.min()))
print("Maximum number of images per species:", int(counts.max()))
print("Average number of images:", float(counts.sum()/325))

Minimum number of images per species: 116
Maximum number of images per species: 249
Average number of images: 145.63692307692307


Since at most we have 249 images per species, we use $n=250$.

In [14]:
n = 250

The formula for Average Precision is the following:

> $AP@n=\frac{1}{GTP}\sum_{k=1}^{n}P@k×rel@k$

where $GTP$ refers to the total number of ground truth positives, $n$ refers to the total number of images we are interested in, $P@k$ refers to the precision@k and $rel@k$ is a relevance function. 

The relevance function is an indicator function which equals 1 if the document at rank $k$ is relevant and equals to 0 otherwise.

In [15]:
def compute_ap(query_index, retrieved_ids):
  query_identity = query_annot['identity'][query_index]
  print(query_index//5, query_identity)
  GTP = len(db_annot.loc[db_annot['identity'] == query_identity])
  relevant = 0
  precision_summation = 0
  for k, id in enumerate(retrieved_ids):
    if db_annot['identity'][id] == query_identity: # relevant result
      relevant = relevant + 1
      precision_at_k = relevant/(k+1)
      precision_summation = precision_summation + precision_at_k
  return (query_identity, precision_summation/GTP)

For each query, $Q$, we can calculate a corresponding $AP$. Then, the $mAP$ is simply the mean of all the queries that were made.
> $mAP = \frac{1}{N}\sum_{i=1}^{N}AP_i$

In our case, $N=325$ (one query per species)

In [16]:
def get_descriptor_from_id(id_object):
  return db_descriptors[id_object]

In [17]:
!rm /content/drive/MyDrive/CV_Birds/performance/feature_extraction/index/FE-no-distr_tree_5kmed_cosine/*
!rm -r /content/drive/MyDrive/CV_Birds/performance/feature_extraction/index/FE-no-distr_tree_5kmed_cosine/tree_structure/*

rm: cannot remove '/content/drive/MyDrive/CV_Birds/performance/feature_extraction/index/FE-no-distr_tree_5kmed_cosine/tree_structure': Is a directory


In [18]:
%cd "/content/drive/MyDrive/CV_Birds/Notebooks/PP-Index"
%run PPIndex.ipynb

/content/drive/.shortcut-targets-by-id/1rI5YNBuaSlCB__w522WEkHjw-nFuvIo0/CV_Birds/Notebooks/PP-Index


#### kmed pivots

In [19]:
kmed_pivots = generate_pivots(db_descriptors, 5, "kMED")

In [20]:
pp_tree = PrefixTree(kmed_pivots, length=3, distance_metric='cosine', base_directory='/content')
if pp_tree.is_empty():
  pp_tree.insert_objects_into_tree(range(len(db_descriptors)))
  pp_tree.save()

[WARNING]: Tree is still empty
Inserting objects into tree_structure ...
--- Insertion into tree tree_structure completed: 48.164 seconds ---


In [21]:
def kmed_pivots_queries(query_index, n):
  start_time = time.time()
  ids, distances = pp_tree.find_nearest_neighbors(query_descriptors[query_index], n)
  end_time = time.time()
  ids = ids.tolist()
  return compute_ap(query_index, ids) + (end_time - start_time,)

In [22]:
aps = []
for query_index in queries_indexes:
  aps.append(kmed_pivots_queries(query_index, n))

 #with concurrent.futures.ThreadPoolExecutor() as executor:
  #average_precisions = [executor.submit(rnd_pivots_queries, query_index, n) for query_index in queries_indexes]
  #for ap in concurrent.futures.as_completed(average_precisions):
   # average_precision = ap.result()
    #aps.append(average_precision)

0 AFRICAN CROWNED CRANE
1 AFRICAN FIREFINCH
2 ALBATROSS
3 ALEXANDRINE PARAKEET
4 AMERICAN AVOCET
5 AMERICAN BITTERN
6 AMERICAN COOT
7 AMERICAN GOLDFINCH
8 AMERICAN KESTREL
9 AMERICAN PIPIT
10 AMERICAN REDSTART
11 ANHINGA
12 ANNAS HUMMINGBIRD
13 ANTBIRD
14 ARARIPE MANAKIN
15 ASIAN CRESTED IBIS
16 BALD EAGLE
17 BALD IBIS
18 BALI STARLING
19 BALTIMORE ORIOLE
20 BANANAQUIT
21 BANDED BROADBILL
22 BANDED PITA
23 BAR-TAILED GODWIT
24 BARN OWL
25 BARN SWALLOW
26 BARRED PUFFBIRD
27 BAY-BREASTED WARBLER
28 BEARDED BARBET
29 BEARDED BELLBIRD
30 BEARDED REEDLING
31 BELTED KINGFISHER
32 BIRD OF PARADISE
33 BLACK & YELLOW bROADBILL
34 BLACK BAZA
35 BLACK FRANCOLIN
36 BLACK SKIMMER
37 BLACK SWAN
38 BLACK TAIL CRAKE
39 BLACK THROATED BUSHTIT
40 BLACK THROATED WARBLER
41 BLACK VULTURE
42 BLACK-CAPPED CHICKADEE
43 BLACK-NECKED GREBE
44 BLACK-THROATED SPARROW
45 BLACKBURNIAM WARBLER
46 BLONDE CRESTED WOODPECKER
47 BLUE COAU
48 BLUE GROUSE
49 BLUE HERON
50 BLUE THROATED TOUCANET
51 BOBOLINK
52 BORNEAN BRI

In [23]:
aps

[('AFRICAN CROWNED CRANE', 0.007910778071129056, 1.8431084156036377),
 ('AFRICAN FIREFINCH', 3.819709702062643e-05, 1.4501879215240479),
 ('ALBATROSS', 0.0007367658020930547, 1.1373357772827148),
 ('ALEXANDRINE PARAKEET', 0.00023773605704182965, 0.4637455940246582),
 ('AMERICAN AVOCET', 0.0006664706458884642, 0.5683128833770752),
 ('AMERICAN BITTERN', 0.004987230839631708, 2.1321403980255127),
 ('AMERICAN COOT', 0.0004990688895176041, 1.138838529586792),
 ('AMERICAN GOLDFINCH', 0.0005113063278864047, 1.6378824710845947),
 ('AMERICAN KESTREL', 0.00022924600152322925, 1.112654685974121),
 ('AMERICAN PIPIT', 0.008989915216280706, 1.1671490669250488),
 ('AMERICAN REDSTART', 0.0003153133310303095, 0.9015011787414551),
 ('ANHINGA', 0.0, 1.1153185367584229),
 ('ANNAS HUMMINGBIRD', 0.026143371017471734, 1.113060712814331),
 ('ANTBIRD', 0.0006974525807459219, 1.6579620838165283),
 ('ARARIPE MANAKIN', 0.03134261991406213, 1.0400333404541016),
 ('ASIAN CRESTED IBIS', 0.0020452544634380074, 0.5295

In [24]:
ap_at_n = np.array([ap[1] for ap in aps])
query_time = np.array(([ap[2] for ap in aps]))

In [25]:
mAP_at_n = np.mean(ap_at_n, axis=0)
avg_query_time = np.mean(query_time, axis=0)
print("mAP:", mAP_at_n)
print("avg. query time: ", avg_query_time)

mAP: 0.004857938319865713
avg. query time:  1.269194462115948


## store results

In [26]:
save_results('/content', "FE-no-distr_tree_5kmed_cosine", aps)

In [27]:
!mv /content/tree_structure* /content/drive/MyDrive/CV_Birds/performance/feature_extraction/index/FE-no-distr_tree_5kmed_cosine/tree_structure/